# Geo Hashing
- [GeoHashing from Scratch in Python](https://www.jtrive.com/posts/geohash-python/index.html)

In [2]:
# !pip install pygeohash
# !pip install folium

In [3]:
import pygeohash

# visualise geohash with folium
import folium 
from folium.features import DivIcon

In [4]:
def get_bbox_geohash(lat, lon, precision=12):
    min_lat, max_lat = -90, 90
    min_lon, max_lon = -180, 180
    for ii in range(5 * precision):
        if ii % 2 == 0:
            # Bisect longitude (E-W).
            mid_lon = (min_lon + max_lon) / 2
            if lon >= mid_lon:     
                min_lon = mid_lon
            else:
                max_lon = mid_lon
        else:
            # Bisect latitude (N-S).
            mid_lat = (min_lat + max_lat) / 2
            if lat >= mid_lat:
                min_lat = mid_lat
            else:
                max_lat = mid_lat
    return [min_lat, min_lon, max_lat, max_lon]


In [39]:
lat, lon, precision = 10.776775578390142, 106.7031296241205, 6
gh_center = pygeohash.encode(latitude=lat, longitude=lon, precision=precision)
min_lat, min_lon, max_lat, max_lon = get_bbox_geohash(lat, lon, precision=precision)

In [40]:
gh_center

'w3gvk1'

In [31]:
# Get mid_lat and mid_lon for GeoHash id placement. 
mid_lat = (min_lat + max_lat) / 2
mid_lon = (min_lon + max_lon) / 2

In [41]:
m = folium.Map(
    location=[lat, lon], 
    #width=900, 
    #height=600, 
    zoom_start=16, 
    zoom_control=True, 
    no_touch=True,
    tiles="OpenStreetMap"
    )

# precision = 6 GeoHash bounding box. 
folium.Rectangle(
    [(min_lat, min_lon), (max_lat, max_lon)], 
    fill_color="red", fill_opacity=.15
    ).add_to(m)

# Red dot at Merchandise Mart. 
folium.CircleMarker(
    location=[lat, lon], radius=5, color="red", fill_color="red", 
    fill_opacity=1
    ).add_to(m)

# precision = 6 GeoHash id.
folium.map.Marker(
    [mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(100,50),
        html=f'<div style="font-size: 20pt">{gh_center}</div>',
        )
    ).add_to(m)
# m

In [37]:
def get_bbox_and_geohash(lat, lon, precision):
    geohash = pygeohash.encode(latitude=lat, longitude=lon, precision=precision)
    min_lat, min_lon, max_lat, max_lon = get_bbox_geohash(lat, lon, precision=precision)
    bbox = [min_lat, min_lon, max_lat, max_lon]
    return geohash, bbox

- Identify GeoHash Neighboring Cells: once the bounding box for the target GeoHash is known we simply increment those coordinates by a small amount, then lookup the GeoHash and bounding box associated with the new coordinate

In [63]:
eps = 1e-10
gh_center, bb_center = get_bbox_and_geohash(lat, lon, precision=precision)
min_lat, min_lon, max_lat, max_lon = bb_center
# Get GeoHash id and bounding box for Northwest cell.
gh_nw, bb_nw = get_bbox_and_geohash(max_lat + eps, min_lon - eps, precision=precision)

# Get GeoHash id and bounding box for Northeast cell.
gh_ne, bb_ne = get_bbox_and_geohash(max_lat + eps, max_lon + eps, precision=precision)

# Get GeoHash id and bounding box for Southeast cell.
gh_se, bb_se = get_bbox_and_geohash(min_lat - eps, max_lon + eps, precision=precision)

# Get GeoHash id and bounding box for Southwest cell.
gh_sw, bb_sw = get_bbox_and_geohash(min_lat - eps, min_lon - eps, precision=precision)

In [64]:
coord_list = zip([gh_center, gh_nw, gh_ne, gh_se, gh_sw],[bb_center, bb_nw, bb_ne, bb_se, bb_sw])

In [65]:
m = folium.Map(
    location=[lat, lon], 
    #width=900, 
    #height=600, 
    zoom_start=16, 
    zoom_control=True, 
    no_touch=True,
    tiles="OpenStreetMap"
)
# Red dot at Merchandise Mart. 
folium.CircleMarker(
    location=[lat, lon], radius=5, color="red", fill_color="red", 
    fill_opacity=1
).add_to(m)

for gh, bb in coord_list:
    min_lat, min_lon, max_lat, max_lon = bb
    # Get mid_lat and mid_lon for GeoHash id placement. 
    mid_lat = (min_lat + max_lat) / 2
    mid_lon = (min_lon + max_lon) / 2
    # precision = 6 GeoHash bounding box. 
    folium.Rectangle(
        [(min_lat, min_lon), (max_lat, max_lon)], 
        fill_color="red", fill_opacity=.15
        ).add_to(m)

    # precision = 6 GeoHash id.
    folium.map.Marker(
        [mid_lat, mid_lon],
        icon=DivIcon(
            icon_size=(250,36),
            icon_anchor=(100,50),
            html=f'<div style="font-size: 20pt">{gh}</div>',
            )
        ).add_to(m)
m